In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.nn.functional as F
import torchvision
import math

from efficientnet_pytorch import EfficientNet

import os
from enum import Enum
from typing import Tuple

import numpy as np
from PIL import Image
from torch.utils.data import Dataset, Subset, random_split, DataLoader, WeightedRandomSampler
from torchvision.transforms import Resize, ToTensor, Normalize, Compose
import torch
import multiprocessing
import random

def cutmix(batch, alpha):
    data, targets = batch

    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)

    image_h, image_w = data.shape[2:]
    cx = np.random.uniform(0, image_w)
    cy = np.random.uniform(0, image_h)
    w = image_w * np.sqrt(1 - lam)
    h = image_h * np.sqrt(1 - lam)
    x0 = int(np.round(max(cx - w / 2, 0)))
    x1 = int(np.round(min(cx + w / 2, image_w)))
    y0 = int(np.round(max(cy - h / 2, 0)))
    y1 = int(np.round(min(cy + h / 2, image_h)))

    data[:, :, y0:y1, x0:x1] = shuffled_data[:, :, y0:y1, x0:x1]
    targets = (targets, shuffled_targets, lam)

    return data, targets


class CutMixCollator:
    def __init__(self, a):
        self.a = a

    def __call__(self, batch):
        batch = torch.utils.data.dataloader.default_collate(batch)
        batch = cutmix(batch, self.alpha)
        return batch


class CutMixCriterion:
    def __init__(self, reduction):
        self.criterion = nn.CrossEntropyLoss(reduction=reduction)

    def __call__(self, preds, targets):
        targets1, targets2, lam = targets
        return lam * self.criterion(
            preds, targets1) + (1 - lam) * self.criterion(preds, targets2)

In [ ]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
seed_everything(42)

In [ ]:
class cfg:
    data_dir = '/opt/ml/input/data/train/'  
    img_dir = f'{data_dir}images'
    df_path = f'{data_dir}train.csv'
    test_dir ='/opt/ml/input/data/eval/'

In [ ]:
means, stds =  [0.56019358, 0.52410121, 0.501457], [0.23318603, 0.24300033, 0.24567522]

In [ ]:
IMG_EXTENSIONS = [
    ".jpg", ".JPG", ".jpeg", ".JPEG", ".png",
    ".PNG", ".ppm", ".PPM", ".bmp", ".BMP",
]

In [ ]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

In [ ]:
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize, GaussianBlur, RandomRotation, ColorJitter, CenterCrop, RandomHorizontalFlip


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


def get_transforms(need=('train', 'val'), img_size=[224, 224], mean =  [0.56019358, 0.52410121, 0.501457], std =[0.23318603, 0.24300033, 0.24567522] ):
    transformations = {}
    if 'train' in need:
        transformations['train'] = transforms.Compose([
            CenterCrop((384, 384)),
            Resize((img_size[0], img_size[1])),
            #RandomHorizontalFlip(p=0.5),
            #RandomRotation([-5, +5]),
            #GaussianBlur(51, (0.1, 2.0)),
            #ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),  # todo : param
            ToTensor(),
            Normalize(mean=mean, std=std),
            #AddGaussianNoise(0., 1.)
        ])
    if 'val' in need:
        transformations['val'] = transforms.Compose([
            CenterCrop((384, 384)),
            Resize((img_size[0], img_size[1])),
            ToTensor(),
            Normalize(mean=mean, std=std),
        ])
    return transformations

In [ ]:
class MaskLabels(int, Enum):
    MASK = 0
    INCORRECT = 1
    NORMAL = 2


class GenderLabels(int, Enum):
    MALE = 0
    FEMALE = 1

    @classmethod
    def from_str(cls, value: str) -> int:
        value = value.lower()
        if value == "male":
            return cls.MALE
        elif value == "female":
            return cls.FEMALE
        else:
            raise ValueError(f"Gender value should be either 'male' or 'female', {value}")


class AgeLabels(int, Enum):
    YOUNG = 0
    MIDDLE = 1
    OLD = 2

    @classmethod
    def from_number(cls, value: str) -> int:
        try:
            value = int(value)
        except Exception:
            raise ValueError(f"Age value should be numeric, {value}")

        if value < 30:
            return cls.YOUNG
        elif value < 60:
            return cls.MIDDLE
        else:
            return cls.OLD

In [ ]:
class MaskBaseDataset(Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1": MaskLabels.MASK,
        "mask2": MaskLabels.MASK,
        "mask3": MaskLabels.MASK,
        "mask4": MaskLabels.MASK,
        "mask5": MaskLabels.MASK,
        "incorrect_mask": MaskLabels.INCORRECT,
        "normal": MaskLabels.NORMAL
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, data_dir, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), val_ratio=0.2):
        self.data_dir = data_dir
        self.mean = mean
        self.std = std
        self.val_ratio = val_ratio

        self.transform = None
        self.setup()
        self.calc_statistics()

    def setup(self):
        profiles = os.listdir(self.data_dir)
        for profile in profiles:
            if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
                continue

            img_folder = os.path.join(self.data_dir, profile)
            for file_name in os.listdir(img_folder):
                _file_name, ext = os.path.splitext(file_name)
                if _file_name not in self._file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
                    continue

                img_path = os.path.join(self.data_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                mask_label = self._file_names[_file_name]

                id, gender, race, age = profile.split("_")
                gender_label = GenderLabels.from_str(gender)
                age_label = AgeLabels.from_number(age)

                self.image_paths.append(img_path)
                self.mask_labels.append(mask_label)
                self.gender_labels.append(gender_label)
                self.age_labels.append(age_label)

    def calc_statistics(self):
        has_statistics = self.mean is not None and self.std is not None
        if not has_statistics:
            print("[Warning] Calculating statistics... It can take a long time depending on your CPU machine")
            sums = []
            squared = []
            for image_path in self.image_paths[:3000]:
                image = np.array(Image.open(image_path)).astype(np.int32)
                sums.append(image.mean(axis=(0, 1)))
                squared.append((image ** 2).mean(axis=(0, 1)))

            self.mean = np.mean(sums, axis=0) / 255
            self.std = (np.mean(squared, axis=0) - self.mean ** 2) ** 0.5 / 255

    def set_transform(self, transform):
        self.transform = transform

    def __getitem__(self, index):
        assert self.transform is not None, ".set_tranform 메소드를 이용하여 transform 을 주입해주세요"

        image = self.read_image(index)
        mask_label = self.get_mask_label(index)
        gender_label = self.get_gender_label(index)
        age_label = self.get_age_label(index)
        multi_class_label = self.encode_multi_class(mask_label, gender_label, age_label)

        image_transform = self.transform(image)
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)

    def get_mask_label(self, index) -> MaskLabels:
        return self.mask_labels[index]

    def get_gender_label(self, index) -> GenderLabels:
        return self.gender_labels[index]

    def get_age_label(self, index) -> AgeLabels:
        return self.age_labels[index]

    def read_image(self, index):
        image_path = self.image_paths[index]
        return Image.open(image_path)

    @staticmethod
    def encode_multi_class(mask_label, gender_label, age_label) -> int:
        return mask_label * 6 + gender_label * 3 + age_label

    @staticmethod
    def decode_multi_class(multi_class_label) -> Tuple[MaskLabels, GenderLabels, AgeLabels]:
        mask_label = (multi_class_label // 6) % 3
        gender_label = (multi_class_label // 3) % 2
        age_label = multi_class_label % 3
        return mask_label, gender_label, age_label

    @staticmethod
    def denormalize_image(image, mean, std):
        img_cp = image.copy()
        img_cp *= std
        img_cp += mean
        img_cp *= 255.0
        img_cp = np.clip(img_cp, 0, 255).astype(np.uint8)
        return img_cp


In [ ]:

transform = get_transforms(mean=means, std=stds)

dataset = MaskBaseDataset(
    data_dir=cfg.img_dir,
    mean=means,
    std=stds
)

n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_set, val_set = random_split(dataset, [n_train, n_val])

train_set.dataset.set_transform(transform['train'])
val_set.dataset.set_transform(transform['val'])

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
dataset = DataLoader(
    dataset,
    num_workers=multiprocessing.cpu_count() // 2,
    shuffle=True,
    pin_memory=use_cuda,
    drop_last=True,)

In [ ]:
collate = CutMixCollator(0.5)

In [ ]:
y_train=[]
for images, labels in train_set:
    labels = int(labels)
    y_train.append(labels)
class_sample_count = [2745, 2050, 415, 3660, 4085, 545, 549, 410, 83, 732, 817, 109, 549, 410, 83, 732, 817, 109] 
weight = 1. / np.array(class_sample_count)
samples_weight = np.array([weight[t] for t in y_train])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)

In [ ]:
train_loader = DataLoader(
    train_set,
    batch_size=64,
    num_workers=multiprocessing.cpu_count() // 2,
    shuffle=True,
    #sampler= sampler,
    collate_fn = collate,
    pin_memory=use_cuda,
    drop_last=True,
)

val_loader = DataLoader(
    val_set,
    batch_size=64,
    num_workers=multiprocessing.cpu_count() // 2,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=True,
)

In [ ]:
class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels=3, num_classes=18):
        super(EfficientNet_MultiLabel, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.network = EfficientNet.from_pretrained('efficientnet-b4', in_channels=self.in_channels, num_classes=self.num_classes)

    def forward(self, x):
        
        x = self.network(x)

        return x

In [ ]:
model = EfficientNet_MultiLabel()

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [ ]:
model.to(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, num):
        self.val = val
        self.sum += val * num
        self.count += num
        self.avg = self.sum / self.count

In [ ]:
import wandb
wandb.init(project='eff_cut')

In [ ]:
LR = 1e-4
EPOCH = 15
criterion_1 = FocalLoss()
#criterion_2 = F1Loss() #CrossEntropyLossWithClassBalancing(num_classes, class_weights) #torch.nn.CrossEntropyLoss # default: cross_entropy
criterion_3 = CutMixCriterion(reduction='mean')
criterion_4 = nn.CrossEntropyLoss()
opt_Adam = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9,0.99))
#scheduler = StepLR(optimizer, args.lr_decay_step, gamma=0.5)
best_val_acc = 0
best_val_loss = np.inf
X=[]
y=[]
#-- Training 
for epoch in range(EPOCH):
    print(epoch)
    # train loop
    model.train()
    loss_value = 0
    matches = 0
    
    loss_meter = AverageMeter()
    accuracy_meter = AverageMeter()
    
    for idx, train_batch in enumerate(train_loader):
        inputs, labels = train_batch
        inputs = inputs.to(device)

        if isinstance(labels, (tuple, list)):
            targets1, targets2, lam = labels
            labels = (targets1.to(device), targets2.to(device), lam)
        
        opt_Adam.zero_grad()

        outs = model(inputs)
    
        loss = criterion_3(outs,labels)#0.4*criterion_1(outs, labels)+0.6*criterion_2(outs, labels) #criterion()(outs, labels)

        loss.backward()
        opt_Adam.step()

        _, preds = torch.max(outs, dim=1)

        loss_ = loss.item()

        num = inputs.size(0)

        if isinstance(labels, (tuple, list)):
            targets1, targets2, lam = labels
            correct1 = preds.eq(targets1).sum().item()
            correct2 = preds.eq(targets2).sum().item()
            accuracy = (lam * correct1 + (1 - lam) * correct2) / num
        else:
            correct_ = preds.eq(targets).sum().item()
            accuracy = correct_ / num
            
        loss_meter.update(loss_, num)
        accuracy_meter.update(accuracy, num)
        #print(loss_meter.avg)
        print(f'Train/Loss: {loss_meter.avg:.2f} ||', f'Train/Acc: {accuracy_meter.avg:.2%}')
        wandb.log({"Train/loss": loss_meter.avg,"Train/acc": accuracy_meter.avg})
    with torch.no_grad():
        model.eval()
        val_loss_items = AverageMeter()
        val_acc_items = AverageMeter()
        figure = None
        
        for val_batch in val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)

            loss_item = criterion_1(outs,labels) #(0.4*criterion_1(outs, labels)+0.6*criterion_2(outs, labels)).item() #criterion()(outs, labels).item()

            _, preds = torch.max(outs, dim=1)

            loss_ = loss_item.item()
            correct_ = preds.eq(labels).sum().item()
            num = inputs.size(0)

            val_loss_items.update(loss_, num)
            val_acc_items.update(correct_, 1)

        accuracy = val_acc_items.sum / len(val_loader.dataset)
        print(f'Val/Loss: {val_loss_items.avg:.2f} ||', f'Val/Acc: {accuracy:.2%}')
        wandb.log({"Val/loss": val_loss_items.avg,"Val/acc": accuracy})

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
submission = pd.read_csv(os.path.join(cfg.test_dir, 'info.csv'))
image_dir = os.path.join(cfg.test_dir, 'images')

In [ ]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    CenterCrop((384, 384)),
    Resize((224, 224), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=means, std=stds),
])
dataset = TestDataset(image_paths, transform)
loader = DataLoader(
    dataset,
    shuffle=False
)

In [ ]:
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
effi_ensemble = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        effi_ensemble.extend(pred.cpu().numpy())
        
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(cfg.test_dir, 'effi_cut.csv'), index=False)
print('test inference is done!')

In [ ]:
vit_preds = pd.read_csv('vit_preds.csv',index_col=0)
vit_preds = vit_preds.to_numpy()

In [ ]:
effi_ensemble = pd.DataFrame(effi_ensemble,columns=range(18))
#effi_ensemble.to_csv('effi_cut_ensemble.csv')
effi_ensemble = effi_ensemble.to_numpy()

In [ ]:
ensemble_prediction = []
pred = 0.4*(vit_preds) + 0.6*effi_ensemble
pred = np.argmax(pred,axis=1)
pred
submission['ans'] = pred
# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(cfg.test_dir, 'vit_effi_cut.csv'), index=False)
print('test inference is done!')